In [11]:
import pandas as pd
from tqdm import *

In [12]:
data_athletes = pd.read_csv('data/athletes.csv')
data_programs = pd.read_csv('data/programs.csv')
data_medal_counts = pd.read_csv('data/medal_counts.csv')
team_df = pd.read_csv('data/team_index.csv')
years_index = range(1896, 2025, 4)
programs_index = pd.read_csv('data/programs_index.csv')['program_id'].tolist()


In [13]:
number_of_years_considered = 3
number_of_years_start = 2
number_each_medal = 329
medal_score = {
    "Gold": 3,
    "Silver": 2,
    "Bronze": 1
}
# team_not_include_in_athletes = []
# team_not_match_in_any_program = []
# team_not_save = []

In [21]:
def make_dataset(noc, team_now, team_sum, predict_data_2028=False):
    tmp_athletes = data_athletes[data_athletes["NOC"] == noc]
    result_df = pd.DataFrame()
    if tmp_athletes.shape[0] == 0:
        # team_not_include_in_athletes.append(noc)
        # print("No such team: " + noc)
        return result_df
        # raise ValueError("No such team: " + team_name + " (" + noc + ")")
    num_participate = 0

    # each year
    if not predict_data_2028:
        tqdm_bar = tqdm(years_index)
    else:
        tqdm_bar = [2024]
    for year in tqdm_bar:
        tmp_one_year = tmp_athletes[tmp_athletes["Year"] == year]

        if tmp_one_year.shape[0] == 0:
            continue
        elif tmp_one_year.shape[0] != 0 and num_participate <= number_of_years_start and not predict_data_2028:
            num_participate += 1
            continue

        if predict_data_2028:
            if noc == "USA":
                host = 1
            else:
                host = 0
        else:
            host = tmp_one_year["Host"].iloc[0]
        tmp_one_year_medal = data_medal_counts[data_medal_counts["Year"] == year]
        sum_g = tmp_one_year_medal["Gold"].sum()
        sum_s = tmp_one_year_medal["Silver"].sum()
        sum_b = tmp_one_year_medal["Bronze"].sum()
        sum_total = tmp_one_year_medal["Total"].sum()

        tmp_one_year_medal = tmp_one_year_medal[tmp_one_year_medal["NOC"] == noc]

        if tmp_one_year_medal.shape[0] != 0:
            rate_g = tmp_one_year_medal["Gold"].sum() / sum_g
            rate_s = tmp_one_year_medal["Silver"].sum() / sum_s
            rate_b = tmp_one_year_medal["Bronze"].sum() / sum_b
            rate_total = tmp_one_year_medal["Total"].sum() / sum_total
        else:
            rate_g = 0
            rate_s = 0
            rate_b = 0
            rate_total = 0

        tmp_df = pd.DataFrame()
        tmp_df["Year"] = [year]
        tmp_df["NOC"] = [noc]
        if not predict_data_2028:
            tmp_df["Gold_rate"] = [rate_g]
            tmp_df["Silver_rate"] = [rate_s]
            tmp_df["Bronze_rate"] = [rate_b]
            tmp_df["Total_rate"] = [rate_total]
        tmp_df["Host"] = [host]

        # each program
        for program in programs_index:
            tmp_one_year_program = tmp_one_year[tmp_one_year["Sport"] == program]
            if tmp_one_year_program.shape[0] == 0:
                tmp_df[program] = [0]
                continue
            tmp_one_year_program_athletes = list(set(tmp_one_year_program["Name"].tolist()))

            team_score_in_program = 0
            athletes_num_in_program = 0

            for each_athletes in tmp_one_year_program_athletes:
                if predict_data_2028:
                    tmp_athletes_record = tmp_athletes[(tmp_athletes["Name"] == each_athletes) & (tmp_athletes["Year"] <= year) & (tmp_athletes["Year"] >= (year - 4 * number_of_years_considered))]
                else:
                    tmp_athletes_record = tmp_athletes[(tmp_athletes["Name"] == each_athletes) & (tmp_athletes["Year"] <= year) & (tmp_athletes["Year"] >= (year - 4 * number_of_years_considered))]

                if tmp_athletes_record.shape[0] == 0:
                    athletes_num_in_program += 1
                    continue
                else:
                    participate_year_list = (list(set(tmp_athletes_record["Year"])))
                    tmp_athletes_score = 0
                    for each_year in participate_year_list:
                        tmp_athletes_score += tmp_athletes_record[tmp_athletes_record["Year"] == each_year]["Medal"].mean()

                    team_score_in_program += tmp_athletes_score / len(participate_year_list)
                    athletes_num_in_program += 1

            # print(year)
            # tmp = data_programs[data_programs["Code"] == program]
            # return tmp["2004"]
            tmp_df[program] = [team_score_in_program]

        result_df = pd.concat([result_df, tmp_df], ignore_index=True)

        if result_df.shape[0] == 0:
            # team_not_match_in_any_program.append(noc)
            return result_df
        if not predict_data_2028:
            tqdm_bar.set_description('Team [{}/{}], NOC: {}, Year: {}, Usable:{} '.format(team_now, team_sum, noc, year, result_df.shape[0]))

        if predict_data_2028:
            break

    result_df.fillna(0, inplace=True)
    return result_df


In [22]:
data_athletes["Medal"] = data_athletes["Medal"].replace("Gold", medal_score["Gold"])
data_athletes["Medal"] = data_athletes["Medal"].replace("Silver", medal_score["Silver"])
data_athletes["Medal"] = data_athletes["Medal"].replace("Bronze", medal_score["Bronze"])
data_athletes["Medal"] = data_athletes["Medal"].replace("No medal", 0)
team_sum = len(team_df)

team_not_include_in_athletes = []
team_not_save = []
team_not_match_in_any_program = []
# make_dataset("CHN", 1, 206)

# Train dataset from each team

In [23]:
print("Make train dataset for each team")
for i in range(team_sum):
    noc = team_df.iloc[i]["NOC"]
    df = make_dataset(noc, i+1, team_sum)
    if df.shape[0] == 0:
        team_not_save.append(noc)
        continue
    df.to_csv("data/dataset/train/" + noc + ".csv", index=False)

Make train dataset for each team


Team [206/206], NOC: LES, Year: 2024, Usable:8 : 100%|██████████| 33/33 [00:00<00:00, 147.81it/s]


# Train dataset from all team

In [24]:
print("Make sum dataset (every country in one dataset)")
sum_dataset_df = pd.DataFrame()
for i in range(team_sum):
    noc = team_df.iloc[i]["NOC"]
    df = make_dataset(noc, i+1, team_sum)
    if df.shape[0] == 0:
        team_not_save.append(noc)
        continue
    sum_dataset_df = pd.concat([sum_dataset_df, df], ignore_index=True)
sum_dataset_df.to_csv("data/dataset/train/sum_dataset.csv", index=False)

Make sum dataset (every country in one dataset)


  0%|          | 0/33 [00:00<?, ?it/s]

Team [206/206], NOC: LES, Year: 2024, Usable:8 : 100%|██████████| 33/33 [00:00<00:00, 139.76it/s]


# Train classify dataset for question 2

In [25]:
classify_df = sum_dataset_df.copy()
classify_df = classify_df.drop(["Gold_rate", "Silver_rate", "Bronze_rate"], axis=1)
classify_df["Total_rate"] = classify_df["Total_rate"].apply(lambda x: 1 if round(x * number_each_medal * 3) >= 1.0 else 0)
classify_df.to_csv("data/dataset/train/classify_dataset.csv", index=False)

# Predict dataset 2028

In [26]:
print("Make predict dataset for each team")
for i in trange(team_sum):
    noc = team_df.iloc[i]["NOC"]
    df = make_dataset(noc, i+1, team_sum, predict_data_2028=True)
    if df.shape[0] == 0:
        team_not_save.append(noc)
        continue
    df.to_csv("data/dataset/predict/" + noc + ".csv", index=False)

Make predict dataset for each team


100%|██████████| 206/206 [00:19<00:00, 10.75it/s]


# Find what teams never get the medal

In [27]:
team_never_get_medal = []
for each_team in team_df["NOC"]:
    team_record = data_medal_counts[data_medal_counts["NOC"] == each_team]
    if team_record.shape[0] == 0:
        team_never_get_medal.append(each_team)
tmp_dict = {"NOC": team_never_get_medal}
pd.DataFrame(tmp_dict).to_csv("data/team_never_get_medal.csv", index=False)